## 2- Preparo dos Dados

#### Validando características

#### Descrição
Base de dados detalhada por NCM: Arquivos CSV com separador ponto e vírgula (;) detalhado por ano, mês, código NCM, código da unidade estatística, código de país de origem do produto, código da UF de destino do produto, código da via de transporte, código da URF de embarque/desembarque, quantidade estatística, quilograma líquido, valor dólar FOB (US$)."

#### Tabela Principal ou simplesmente 'a tabela'

In [65]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import seaborn as sns
#import matplotlib.pyplot as plt

i=pd.read_csv('~/comex/data/imp1.csv')
i.info()

A tabela não apresenta valores nulos.

In [67]:
new_names=['ANO', 'MES', 'NCM', 'UNID', 'PAIS', 'UF', 'VIA','URF', 'QT', 'KG', 'FOB', 'FRETE', 'SEGURO']

i.columns=new_names

print(f"\nOs nomes das colunas foram simplificados para:\n\n{new_names}\n\n")


Os nomes das colunas foram simplificados para:

['ANO', 'MES', 'NCM', 'UNID', 'PAIS', 'UF', 'VIA', 'URF', 'QT', 'KG', 'FOB', 'FRETE', 'SEGURO']




> Pela descrição, as 8 primeiras colunas são referentes a características que uma importação pode ter.

In [68]:
i[i.columns[:8]]=i[i.columns[:8]].astype('category')

In [69]:
print(f"\nÉ {i.select_dtypes(include='category').duplicated().any()} a existência de combinações de categorias em duplicidade.")



É False a existência de combinações de categorias em duplicidade.


#### Colunas de tempo: ANO e MÊS

In [70]:
a=(i==i.sort_values(by=['ANO','MES'])).all().all()
if a==True:
    print('\nA tabela esta ordenada por data.')
else: 
    print('\nA tabela esta não esta ordenada por data desde de a origem e\
    todos os relatórios a seguir estão sobre suspeita de erro.\n')


A tabela esta ordenada por data.


In [71]:
print(f"\n primeira data é o mês {i.iloc[0,0:2][1]} de {i.iloc[0,0:2][0]}.\n")


 primeira data é o mês 1 de 1997.



In [72]:
print(f"\nA última data é o mês {i.iloc[-1,0:2][1]} de {i.iloc[-1,0:2][0]}.\n")


A última data é o mês 10 de 2023.



In [73]:
if i.MES.iloc[-1]!=12:
    if (i[['ANO','MES']].drop_duplicates().groupby(by='ANO').MES.count()[:-1]!=12).sum()==0:
        print("\nCom exceção do último ano, que esta correndo, todos os anos apresentam 12 meses.\n")
    else:
        print("\nHá anos incompletos de meses nos anos anteriores ao corrente.\n")
else:
    if (i[['ANO','MES']].drop_duplicates().groupby(by='ANO').MES.count()[:-1]!=12).sum()==0:
        print("\nTodos os anos apresentam 12 meses.\n")
    else:
        print("\nHá anos incompletos de meses.\n")


Com exceção do último ano, que esta correndo, todos os anos apresentam 12 meses.



>  As colunas de características serão abordadas uma a uma, porém, a ordem de abordagem será em função da lógica de filtragem dos dados.

#### URF

Esse bloco de notas mantém o foco em empresas que tenham interesse em importações específicamente pelos portos próximos a capital, Salvador, na Bahia.

Segundo a [CODEBA](https://www.codeba.gov.br/eficiente/sites/portalcodeba/pt-br/home.php), os portos dessa região são o de ARATU-CANDEIAS e o porto de SALVADOR.

Os códigos que representam as unidades dada Receita Federal (URF) que atendem estes portos são:

In [74]:
urf=pd.read_csv('~/comex/metadata/URF.csv', sep=';', encoding='Latin-1')
portos=['ARATU', 'CANDEIAS', 'SALVADOR']

q=pd.DataFrame()
for n in portos:
    urf_t= urf[urf.NO_URF.str.contains(n)]
    q=pd.concat([q,urf_t])
urf_f1=q.CO_URF.tolist()
urf_f1

[517602, 517600, 517700, 510100, 517800, 517801]

In [75]:
i=i[i.URF.isin(urf_f1)]

Porém, duas das URFs listadas movimentaram cargas milhares de vezes menor do que as demais URFs. 
Sendo assim, essas cargas e URFs serão descartadas.

In [76]:
list_urf_del=i.groupby(by='URF').KG.mean().sort_values().reset_index().head(2).URF.tolist()
i=i[~i.URF.isin(list_urf_del)]

#### UNID e QT

Essas duas colunas representam **unidade de medida** e **peso** utilizadas no país de origem, que pode ser  diferente da unidade de peso (KG), que é a principal. Essa essas unidades de origem não são do interesse desse estudo e , por isso, serão eliminadas.

In [77]:
i=i.drop(columns=['QT','UNID'])

#### UF

> [Unidades Federativas do Brasil](https://pt.wikipedia.org/wiki/Unidades_federativas_do_Brasil)

In [78]:
print(f"\nDepois das filtragens das unidades da receita federal, ainda se constata que {len(i.UF.unique())} UF ainda se mantém na tabela.\nPorém, a seguir, constatamos que o estado da  Bahia consome praticamente todo o volume importado, sendo o envio para outros estado insignificantes. Por isso, as importações das demais UFs serão descartadas, assim como será \ndescartada a própria coluna UF.\n")


Depois das filtragens das unidades da receita federal, ainda se constata que 18 UF ainda se mantém na tabela.
Porém, a seguir, constatamos que o estado da  Bahia consome praticamente todo o volume importado, sendo o envio para outros estado insignificantes. Por isso, as importações das demais UFs serão descartadas, assim como será 
descartada a própria coluna UF.



In [79]:
print(i.groupby(by='UF').KG.sum().sort_values(ascending=False).head())
i=i[i.UF=='BA']
i=i.drop(columns='UF')

UF
BA    21688912912
SE      391165441
AL      153240775
GO      113141516
TO       55558994
Name: KG, dtype: int64


#### Coluna Via

A coluna 'VIA' representa o meio de transporte utilizado, que pode ser marítimo, terrestre, e outros.

O intersse desse bloco de notas é smente em produtos via os portos citados e, portanto, a via maritma será mantida e as outras descartadas, bem como será descartada a coluna VIA, por ficar sem função.

In [80]:
i=i[i.VIA==1]
i=i.drop(columns='VIA')

#### PAÍS

In [81]:
print(f"\n\n\n\n{len(i.PAIS.unique())} países estão presentes como exportadores.\n\n\n")





58 países estão presentes como exportadores.





#### NCM

A coluna e o termo NCM é referente a [Nomenclatura Comum do Mercosul](www.google.com) e, no caso de fertilizante, define a composição química e, a princípio, será utilizada para identificar um único produto.

In [82]:
print(f"\nA tabela apresenta {len(i.NCM.unique())} códigos NCM, que a princípio consideramos produtos únicos. \
\n\nPorém, a 'Contratante' busca valorizar o uso de sua capacidade de movimentação de produto, \n\
e por isso deseja focar esforço sobre os 6 produtos (NCM) que mais movimentaram valor CIF no ano {i.ANO.iloc[-1]-1}.\n")


A tabela apresenta 29 códigos NCM, que a princípio consideramos produtos únicos. 

Porém, a 'Contratante' busca valorizar o uso de sua capacidade de movimentação de produto, 
e por isso deseja focar esforço sobre os 6 produtos (NCM) que mais movimentaram valor CIF no ano 2022.



In [83]:
i=i.assign(CIF=(i.FOB+i.FRETE+i.SEGURO))

In [84]:
ai=i.ANO.drop_duplicates().iloc[-7]

In [85]:
af=i.ANO.drop_duplicates().iloc[-2]

In [86]:
anos_considerados=np.arange(ai,af)

In [87]:
top_ncm=i[i.ANO.isin(anos_considerados)].groupby(by='NCM').CIF.sum().reset_index().sort_values(by='CIF', ascending=False).head(6).NCM.tolist()
top_ncm

[31042090, 31054000, 31021010, 31055900, 31022100, 31059090]

In [88]:
i=i[i.NCM.isin(top_ncm)].drop(columns='CIF')

In [89]:
NCM_f1=i['NCM'].unique().tolist()

In [90]:
i.NCM=i.NCM.cat.set_categories(NCM_f1)

In [91]:
ncm=pd.read_csv('~/comex/metadata/NCM.csv',sep=';',encoding='Latin-1')
i.groupby(by='NCM').KG.sum().reset_index().merge(ncm, right_on='CO_NCM', left_on='NCM').head(10).rename(columns={'NO_NCM_POR':'NCM foco'}).sort_values(by='KG',ascending=False)[['NCM','NCM foco']].reset_index(drop=True)

,NCM,NCM foco
0,31042090,Outros cloretos de potássio
1,31054000,Diidrogeno-ortofosfato de amônio (fosfato mono...
2,31021010,"Ureia, mesmo em solução aquosa, com teor de ni..."
3,31022100,Sulfato de amônio
4,31055900,"Outros adubos/fertilizantes minerais químicos,..."
5,31059090,Outros adubos/fertilizantes minerais químicos ...


In [92]:
i.to_csv('~/comex/data/imp2.csv', index=False)